In [1]:
import json
import numpy
from pprint import pprint

with open('/Users/elena/publicCodeRepos/overcode_data/iterPower-edx/output/solutions.json') as solutions_file:    
    solutions = json.load(solutions_file)

In [2]:
with open('/Users/elena/publicCodeRepos/overcode_data/iterPower-edx/output/variables.json') as vars_file:    
    solvars = json.load(vars_file)

In [4]:
doc_matrix = numpy.zeros((len(solutions),len(solvars)))

for i,varIds in enumerate([sol['variableIDs'] for sol in solutions]):
    print i, varIds
    for var_id in varIds:
        doc_matrix[i,var_id-1] += 1

0 [1, 2, 3, 4]
1 [2, 4, 5, 6]
2 [2, 3, 7]
3 [2, 3, 7]
4 [2, 3, 7]
5 [2, 3, 7]
6 [2, 3, 7]
7 [2, 3, 7]
8 [2, 3, 7]
9 [2, 3, 7]
10 [8, 9, 10]
11 [2, 3, 4, 7]
12 [3, 2, 11, 4]
13 [2, 3, 4, 7]
14 [2, 3, 4, 7]
15 [2, 3, 7]
16 [2, 3, 7]
17 [2, 12, 10]
18 [2, 3, 7]
19 [2, 3, 4, 13]
20 [2, 3, 4, 14]
21 [2, 3, 7]
22 [7, 12, 15]
23 [2, 3, 7]
24 [2, 3, 7]
25 [3, 2, 11, 4]
26 [2, 3, 7]
27 [16, 2, 10]
28 [17, 2]
29 [2, 3, 7]
30 [10, 12, 15]
31 [16, 2, 10]
32 [17, 2, 3]
33 [2, 3, 7]
34 [2, 3, 7]
35 [18, 19, 4, 2]
36 [2, 3, 4, 7]
37 [2, 3, 7]
38 [2, 3, 4, 7]
39 [12, 2, 20, 4]
40 [2, 21, 7]
41 [2, 3, 7]
42 [16, 2, 10]
43 [3, 2, 11, 4]
44 [2, 10, 6]
45 [16, 2, 10]
46 [16, 10, 22]
47 [2, 3, 4, 7]
48 [16, 4, 2, 20]
49 [2, 3, 7]
50 [24, 4, 12, 23]
51 [2, 3, 7]
52 [25, 2, 12]
53 [2, 3, 7]
54 [16, 2, 10]
55 [26, 10, 2]
56 [3, 2, 11, 4]
57 [26, 10, 4, 2]
58 [4, 2, 27, 28]
59 [2, 3, 7]
60 [2, 12, 10]
61 [2, 3, 7]
62 [2, 4, 29, 30]
63 [2, 3, 4, 14]
64 [32, 10, 2, 31]
65 [2, 12, 10]
66 [2, 12, 10]
67 [2, 3, 7]


In [5]:
matrix = doc_matrix

"""
(C) Mathieu Blondel - 2010
License: BSD 3 clause

Implementation of the collapsed Gibbs sampler for
Latent Dirichlet Allocation, as described in

Finding scientifc topics (Griffiths and Steyvers)
"""

import numpy as np
import scipy as sp
from scipy.special import gammaln

def sample_index(p):
    """
    Sample from the Multinomial distribution and return the sample index.
    """
    return np.random.multinomial(1,p).argmax()

def word_indices(vec):
    """
    Turn a document vector of size vocab_size to a sequence
    of word indices. The word indices are between 0 and
    vocab_size-1. The sequence length is equal to the document length.
    """
    for idx in vec.nonzero()[0]:
        for i in xrange(int(vec[idx])):
            yield idx

def log_multi_beta(alpha, K=None):
    """
    Logarithm of the multinomial beta function.
    """
    if K is None:
        # alpha is assumed to be a vector
        return np.sum(gammaln(alpha)) - gammaln(np.sum(alpha))
    else:
        # alpha is assumed to be a scalar
        return K * gammaln(alpha) - gammaln(K*alpha)

class LdaSampler(object):

    def __init__(self, n_topics, alpha=0.1, beta=0.1):
        """
        n_topics: desired number of topics
        alpha: a scalar (FIXME: accept vector of size n_topics)
        beta: a scalar (FIME: accept vector of size vocab_size)
        """
        self.n_topics = n_topics
        self.alpha = alpha
        self.beta = beta

    def _initialize(self, matrix):
        n_docs, vocab_size = matrix.shape

        # number of times document m and topic z co-occur
        self.nmz = np.zeros((n_docs, self.n_topics))
        # number of times topic z and word w co-occur
        self.nzw = np.zeros((self.n_topics, vocab_size))
        self.nm = np.zeros(n_docs)
        self.nz = np.zeros(self.n_topics)
        self.topics = {}

        for m in xrange(n_docs):
            # i is a number between 0 and doc_length-1
            # w is a number between 0 and vocab_size-1
            #print m, matrix[m, :] #, word_indices(matrix[m, :])
            for i, w in enumerate(word_indices(matrix[m, :])):
                # choose an arbitrary topic as first topic for word i
                #print 'i,w',i,w
                z = np.random.randint(self.n_topics)
                self.nmz[m,z] += 1
                self.nm[m] += 1
                self.nzw[z,w] += 1
                self.nz[z] += 1
                self.topics[(m,i)] = z

    def _conditional_distribution(self, m, w):
        """
        Conditional distribution (vector of size n_topics).
        """
        vocab_size = self.nzw.shape[1]
        left = (self.nzw[:,w] + self.beta) / \
               (self.nz + self.beta * vocab_size)
        right = (self.nmz[m,:] + self.alpha) / \
                (self.nm[m] + self.alpha * self.n_topics)
        #print 'left',left
        #print 'right',right
        #print 'sizes',left.shape,right.shape
        p_z = left * right
        # normalize to obtain probabilities
        p_z /= np.sum(p_z)
        #print 'p_z',p_z
        return p_z

    def loglikelihood(self):
        """
        Compute the likelihood that the model generated the data.
        """
        vocab_size = self.nzw.shape[1]
        n_docs = self.nmz.shape[0]
        lik = 0

        for z in xrange(self.n_topics):
            lik += log_multi_beta(self.nzw[z,:]+self.beta)
            lik -= log_multi_beta(self.beta, vocab_size)

        for m in xrange(n_docs):
            lik += log_multi_beta(self.nmz[m,:]+self.alpha)
            lik -= log_multi_beta(self.alpha, self.n_topics)

        return lik

    def phi(self):
        """
        Compute phi = p(w|z).
        """
        V = self.nzw.shape[1]
        num = self.nzw + self.beta
        num /= np.sum(num, axis=1)[:, np.newaxis]
        return num

    def run(self, matrix, maxiter=10): #2 used to be 30 TODO: change it back
        """
        Run the Gibbs sampler.
        """
        n_docs, vocab_size = matrix.shape
        self._initialize(matrix)

        for it in xrange(maxiter):
            #print it,
            for m in xrange(n_docs):
                #print 'it, m, matrix[m, :]',it, m, matrix[m, :]
                for i, w in enumerate(word_indices(matrix[m, :])):
                    #print 'iteration, doc, index, word',it, m, i, w
                    z = self.topics[(m,i)]
                    self.nmz[m,z] -= 1
                    self.nm[m] -= 1
                    self.nzw[z,w] -= 1
                    self.nz[z] -= 1

                    #print 'prev z',z
                    p_z = self._conditional_distribution(m, w)
                    z = sample_index(p_z)
                    #print 'new z',z
                    #print
                    #raw_input("Press Enter to continue...")

                    self.nmz[m,z] += 1
                    self.nm[m] += 1
                    self.nzw[z,w] += 1
                    self.nz[z] += 1
                    self.topics[(m,i)] = z

            # FIXME: burn-in and lag!
            yield self.phi()

print 'starting!'

N_TOPICS = 100
DOCUMENT_LENGTH = matrix.shape[1]

sampler = LdaSampler(N_TOPICS)

liklihood_over_iterations = []
phi_over_iterations = []
for it, phi in enumerate(sampler.run(matrix)):
    liklihood = sampler.loglikelihood()
    liklihood_over_iterations.append(liklihood)
    phi_over_iterations.append(phi)
    if it % 1 == 0:
        print "Iteration", it
        print "Likelihood", liklihood

starting!
Iteration 0
Likelihood -25973.1622236
Iteration 1
Likelihood -25238.3172061
Iteration 2
Likelihood -24865.828697
Iteration 3
Likelihood -24712.3581499
Iteration 4
Likelihood -24478.8360719
Iteration 5
Likelihood -24358.742438
Iteration 6
Likelihood -24346.2934184
Iteration 7
Likelihood -24327.2601999
Iteration 8
Likelihood -24183.6391394
Iteration 9
Likelihood -24201.6992409
